# GEM ML Framework Demonstrator - Water Segmentation
In these notebooks, we will get a feeling of how the GEM ML framework can be used for the segmentation of water bodies using Sentinel-1 imagery as input and Sentinel-2 based normalized difference water index (NDWI) as a reference.
The idea is to use a neural network (NN) model for the analysis.
Thanks to the flexibility of the GEM ML framework, the model used can be replaced by changing the configuration only.
We will have a look at the following notebooks separately:
- 00_Configuration
- 01_DataAcquisition
- 02_DataNormalization
- 03_TrainingValidationTesting
- 04_PyTorchTasks_ModelForwardTask

by Michael Engel (m.engel@tum.de)

-----------------------------------------------------------------------------------

# Data Acquisition
Here, we define our `EOWorkflow` for the download of our desired data.

In [ ]:
import datetime as dt
import os

import folium
import geopandas as gpd
import numpy as np

from folium import plugins as foliumplugins
from sentinelhub import DataCollection, UtmZoneSplitter
from shapely.geometry import Polygon

from libs.ConfigME import Config
from tasks.TDigestTask import TDigestTask
from tasks.PickIdxTask import PickIdxTask
from tasks.SaveValidTask import SaveValidTask

from eolearn.core import EOExecutor, EONode, EOWorkflow, FeatureType, MapFeatureTask, MergeEOPatchesTask, OverwritePermission, ZipFeatureTask
from eolearn.io import SentinelHubInputTask, SentinelHubEvalscriptTask, get_available_timestamps
from eolearn.mask import JoinMasksTask


print("Working Directory:", os.getcwd())

# Config
First, we load our configuration file which provides all information we need throughout the script.

In [ ]:
#%% load configuration file
config = Config.LOAD("config.dill")

# Area of Interest
Let's load the geojson of our area of interests for training, validation and testing, respectively.

In [ ]:
#%% load geojson files
aoi_train = gpd.read_file(config['AOI_train'])
aoi_validation = gpd.read_file(config['AOI_validation'])
aoi_test = gpd.read_file(config['AOI_test'])

#%% find best suitable crs and transform to it
crs_train = aoi_train.estimate_utm_crs()
aoi_train = aoi_train.to_crs(crs_train)
aoi_train = aoi_train.buffer(config['AOIbuffer'])

crs_validation = aoi_validation.estimate_utm_crs()
aoi_validation = aoi_validation.to_crs(crs_validation)
aoi_validation = aoi_validation.buffer(config['AOIbuffer'])

crs_test = aoi_test.estimate_utm_crs()
aoi_test = aoi_test.to_crs(crs_test)
aoi_test = aoi_test.buffer(config['AOIbuffer'])

#%% dict for query
aois = {"train":aoi_train,
        "validation":aoi_validation,
        "test":aoi_test}

Since our **area of interests are too large**, we **split** them into a set of smaller bboxes.

In [ ]:
#%% calculate and print size
aoi_train_shape = aoi_train.geometry.values[0]
aoi_train_width = aoi_train_shape.bounds[2]-aoi_train_shape.bounds[0]
aoi_train_height = aoi_train_shape.bounds[3]-aoi_train_shape.bounds[1]
print(f"Dimension of the training area is {aoi_train_width:.0f} x {aoi_train_height:.0f} m2")
aoi_validation_shape = aoi_validation.geometry.values[0]
aoi_validation_width = aoi_validation_shape.bounds[2]-aoi_validation_shape.bounds[0]
aoi_validation_height = aoi_validation_shape.bounds[3]-aoi_validation_shape.bounds[1]
print(f"Dimension of the validation area is {aoi_validation_width:.0f} x {aoi_validation_height:.0f} m2")
aoi_test_shape = aoi_test.geometry.values[0]
aoi_test_width = aoi_test_shape.bounds[2]-aoi_test_shape.bounds[0]
aoi_test_height = aoi_test_shape.bounds[3]-aoi_test_shape.bounds[1]
print(f"Dimension of the test area is {aoi_test_width:.0f} x {aoi_test_height:.0f} m2")

#%% create a splitter to obtain a list of bboxes
bbox_splitter_train = UtmZoneSplitter([aoi_train_shape], aoi_train.crs, config["patchpixelwidth"]*config["resolution"])
bbox_splitter_validation = UtmZoneSplitter([aoi_validation_shape], aoi_validation.crs, config["patchpixelwidth"]*config["resolution"])
bbox_splitter_test = UtmZoneSplitter([aoi_test_shape], aoi_test.crs, config["patchpixelwidth"]*config["resolution"])

bbox_list_train = np.array(bbox_splitter_train.get_bbox_list())
info_list_train = np.array(bbox_splitter_train.get_info_list())
bbox_list_validation = np.array(bbox_splitter_validation.get_bbox_list())
info_list_validation = np.array(bbox_splitter_validation.get_info_list())
bbox_list_test = np.array(bbox_splitter_test.get_bbox_list())
info_list_test = np.array(bbox_splitter_test.get_info_list())

#%% dict for query
bbox_lists = {"train":bbox_list_train,
              "validation":bbox_list_validation,
              "test":bbox_list_test}
info_lists = {"train":info_list_train,
              "validation":info_list_validation,
              "test":info_list_test}

The **bbox list would be sufficient** for starting the training procedure using eo-learn.
To check if we muddled up something, however, we want to visualize it!
Since our area of interest is rather large, we face the problem of multiple coordinate refernce systems.
Unfortunately, **geopandas does not support multiple crs in one dataframe** as described [here](https://github.com/sentinel-hub/sentinelhub-py/issues/123).
Hence, we have to define a set of tiles for each separately.

In [ ]:
tiles = []
crss_uniques = []
for _ in ["train","validation","test"]:
    tiles.append([])
    #%% determine number of coordinate reference systems
    crss = [bbox_._crs for bbox_ in bbox_lists[_]]
    crss_unique = np.array(list(dict.fromkeys(crss)))
    crss_uniques.append(crss_unique)
    n_crss = len(crss_unique)

    #%% sort geometries and indices by crs and store to disk
    geometries = [[] for i in range(n_crss)]
    idxs = [[] for i in range(n_crss)]
    idxs_x = [[] for i in range(n_crss)]
    idxs_y = [[] for i in range(n_crss)]
    for i,info in enumerate(info_lists[_]):
        idx_ = np.argmax(crss_unique==bbox_lists[_][i]._crs)

        geometries[idx_].append(Polygon(bbox_lists[_][i].get_polygon())) # geometries sorted by crs
        idxs[idx_].append(info["index"]) # idxs sorted by crs
        idxs_x[idx_].append(info["index_x"]) # idxs_x sorted by crs
        idxs_y[idx_].append(info["index_y"]) # idxs_y sorted by crs

    for i in range(n_crss):
        #%% build dataframe of our areas of interest (and each crs)
        tiles[-1].append(
            gpd.GeoDataFrame(
                {"index": idxs[i], "index_x": idxs_x[i], "index_y": idxs_y[i]},
                crs="EPSG:"+crss_unique[i]._value_,
                geometry=geometries[i]
            )
        )
        #%%% save dataframes to shapefiles
        tiles[-1][-1].to_file(os.path.join(config["dir_results"],f"grid_aoi_{_}_{i}_EPSG{str(crss_unique[i]._value_)}.gpkg"), driver="GPKG")

We have sorted the tiles according to their corresponding crs.
Now we want to visualize it in a nice map.
Here, it is important to **reproject the tiles** to the crs of our **mapping application** - we do that only for this purpose, the **bbox list is not affected** by this.

In [ ]:
#%% print amount of patches
print("Total number of tiles:",[len(bbox_list) for bbox_list in bbox_lists.values()])

#%% visualize using folium
aoi_folium = aoi_validation.to_crs("EPSG:4326") # use validation for visualisation
location = [aoi_folium.centroid.y,aoi_folium.centroid.x]

mapwindow = folium.Map(location=location, tiles='Stamen Terrain', zoom_start=6)

colors = ["blue","green","red"]
for i,_ in enumerate(["train","validation","test"]):
    #%%% add aois
    #%%%% train
    mapwindow.add_child(
        folium.features.Choropleth(
            aois[_].to_crs("EPSG:4326").to_json(),
            fill_color=colors[i],
            nan_fill_color=colors[i],
            fill_opacity=0,
            nan_fill_opacity=0.5,
            line_color=colors[i],
            line_weight=1,
            line_opacity=0.6,
            smooth_factor=5,
            name=f"{_} area"
        )
    )

    #%%% add grids in blue color
    for t_,tiles_ in enumerate(tiles[i]):
        cp = folium.features.Choropleth(
                tiles_.to_crs("EPSG:4326").to_json(),
                fill_color=colors[i],
                nan_fill_color="black",
                fill_opacity=0,
                nan_fill_opacity=0.5,
                line_color=colors[i],
                line_weight=0.5,
                line_opacity=0.6,
                smooth_factor=5,
                name=f"{_} grid EPSG:{crss_uniques[i][t_]._value_}"
            ).add_to(mapwindow)

        # display index next to cursor
        folium.GeoJsonTooltip(
            ['index'],
            aliases=['Index:'],
            labels=False,
            style="background-color:rgba(0,101,189,0.4); border:2px solid white; color:white;",
            ).add_to(cp.geojson)

#%%% add some controls
folium.LayerControl().add_to(mapwindow)
foliumplugins.Fullscreen(force_separate_button=True).add_to(mapwindow)

#%%% save, render and display
mapwindow.save(os.path.join(config["dir_results"],'gridmap.html'))
mapwindow.render()
mapwindow

# Input Tasks
Now, it is time to define some input tasks for our `eo-learn` workflows.
As an input, we will take a [Sentinel-Hub-Input-Task](https://eo-learn.readthedocs.io/en/latest/eolearn.io.sentinelhub_process.html#eolearn.io.sentinelhub_process.SentinelHubInputTask) for querying **Sentinel-1 data**.
As a reference, we will use a [Sentinel-Hub-Evalscript-Task](https://eo-learn.readthedocs.io/en/latest/eolearn.io.sentinelhub_process.html#eolearn.io.sentinelhub_process.SentinelHubEvalscriptTask) with Sentinel-2 data to **calculate the NDWI**.
For **cloud masking** we use the mask calculated by S2Cloudless.

The date of our reference observations by Sentinel-2 define the time intervals for our input.
If one acquisition of Sentinel-2 has been on the 2022-10-07, for example, our input is the Sentinel-1 observation closest to that date.

In [ ]:
#%% Sentinel-Hub-Input-Task
task_data = SentinelHubInputTask(
    data_collection = DataCollection.SENTINEL1_IW,
    size = None,
    resolution = config["resolution"],
    bands_feature = (FeatureType.DATA, "data"),
    bands = None,
    additional_data = (FeatureType.MASK, "dataMask", "dmask_data"),
    evalscript = None,
    maxcc = None,
    time_difference = dt.timedelta(hours=1),
    cache_folder = config["dir_cache"],
    max_threads = config["threads"],
    config = config["SHconfig"],
    bands_dtype = np.float32,
    single_scene = False,
    mosaicking_order = "mostRecent",
    aux_request_args = None
)

In order to get the closest observation with respect to our observation date, we pick the last one only.

In [ ]:
#%% Pick-Idx-Task
task_pick = PickIdxTask(
    in_feature = (FeatureType.DATA, "data"),
    out_feature = None, # None for replacing in_feature
    idx = [[-1],...] # -1 in brackets for keeping dimensions of numpy array
)

For the normalization of our dataset we will use the T-Digest algorithm.
It is designed for quantile approximation close to the tails which we need for the common quantile scaler in the realm of ML.

In [ ]:
#%% T-Digest-Task
task_tdigest = TDigestTask(
    in_feature = (FeatureType.DATA, 'data'),
    out_feature = (FeatureType.SCALAR_TIMELESS, 'tdigest_data'),
    mode = None,
    pixelwise = False
)

# Reference Task
The reference is calculated using some thresholded value of the NDWI.
To enable the user to use other thresholds after downloading the patches, the raw NDWI and the corresponding bands will be stored within the `EOPatch` as well.
Additionally, we will download the RGB bands for visualisation purposes.

In [ ]:
#%% Sentinel-Hub-Evalscript-Task
task_reference = SentinelHubEvalscriptTask(
    features = [(FeatureType.DATA_TIMELESS,"B02"),
                (FeatureType.DATA_TIMELESS,"B03"),
                (FeatureType.DATA_TIMELESS,"B04"),
                (FeatureType.DATA_TIMELESS,"B08"),
                (FeatureType.DATA_TIMELESS,"B8A"),
                (FeatureType.DATA_TIMELESS,"B11"),
                (FeatureType.DATA_TIMELESS,"NDWI"),
                (FeatureType.MASK_TIMELESS,"reference"),
                (FeatureType.MASK_TIMELESS,"dmask_reference"),
                (FeatureType.MASK_TIMELESS,"cmask_reference")
               ],
    evalscript = """
        //VERSION=3
        
        // evaluation function
        function evaluatePixel(samples) {
            // calculate NDWI
            let val = index(samples.B03,samples.B8A);
            
            // classify water
            let water = 0;
            if (val>0.1){
                water = 1;
            } else {
                water = 0;
            }
            
            // return water, dataMask and cloudMask
            return {
                B02: [samples.B02],
                B03: [samples.B03],
                B04: [samples.B04],
                B08: [samples.B08],
                B8A: [samples.B8A],
                B11: [samples.B11],
                NDWI: [val],
                reference: [water],
                dmask_reference: [samples.dataMask],
                cmask_reference: [!samples.CLM]
            };
        }

        //setup function
        function setup() {
            return {
                input: [{
                    bands:[
                        "B02",
                        "B03",
                        "B04",
                        "B08",
                        "B8A",
                        "B11",
                        "dataMask",
                        "CLM"
                    ],
                    units:"DN",
                }],
                output:[
                    // output definition of bands
                    {
                        id: "B02",
                        bands: 1,
                        sampleType: SampleType.UINT16
                    },
                    {
                        id: "B03",
                        bands: 1,
                        sampleType: SampleType.UINT16
                    },
                    {
                        id: "B04",
                        bands: 1,
                        sampleType: SampleType.UINT16
                    },
                    {
                        id: "B08",
                        bands: 1,
                        sampleType: SampleType.UINT16
                    },
                    {
                        id: "B8A",
                        bands: 1,
                        sampleType: SampleType.UINT16
                    },
                    {
                        id: "B11",
                        bands: 1,
                        sampleType: SampleType.UINT16
                    },
                    
                    // output definition of NDWI
                    {
                        id: "NDWI",
                        bands: 1,
                        sampleType: SampleType.FLOAT32
                    },
                
                    // output definition of water label
                    {
                        id: "reference",
                        bands: 1,
                        sampleType: SampleType.UINT8,
                    },
                    
                    // output definition of dataMask
                    {
                        id: "dmask_reference",
                        bands: 1,
                        sampleType: SampleType.UINT8,
                    },
                    
                    // output definition of cloudMask
                    {
                        id: "cmask_reference",
                        bands: 1,
                        sampleType: SampleType.UINT8,
                    },
                ]
            }
        }
    """,
    data_collection = DataCollection.SENTINEL2_L1C,
    size = None,
    resolution = config["resolution"],
    maxcc = config["maxcc"],
    time_difference = dt.timedelta(hours=1),
    cache_folder = config["dir_cache"],
    max_threads = config["threads"],
    config = config["SHconfig"],
    mosaicking_order = "mostRecent",
    aux_request_args = None
)

## Masking
These EOTasks define the data we want to have as an input and as a reference for our learning problem.
Still, **we have areas not providing data** at all or not in a meaningful way as for clouds, for example.
That is, we take care of our dataMasks and the cloudMask for the reference.

For the sake of simplicity we want to **filter out every sample of the input not providing the full data**.
This filtering will be done based on the analysis of the [MapFeatureTask](https://eo-learn.readthedocs.io/en/latest/_modules/eolearn/core/core_tasks.html#MapFeatureTask) applied to the dataMask.

In [ ]:
#%% Filter out incomplete input data patches
def checker_nodata(array):
    return bool(np.all(array))
task_data_check = MapFeatureTask(
    input_features = (FeatureType.MASK, "dmask_data"),
    output_features = (FeatureType.META_INFO, "valid_data"),
    map_function = checker_nodata
)

**Regarding the reference**, we want to filter out all data not providing **at least 5% water body coverage**.
Again, this will be done based on a [MapFeatureTask](https://eo-learn.readthedocs.io/en/latest/_modules/eolearn/core/core_tasks.html#MapFeatureTask).

In [ ]:
#%% Filter out no water patches
def checker_waterdata(array):
    return bool(np.count_nonzero(array)>=0.05*array.size)
task_reference_check = MapFeatureTask(
    input_features = (FeatureType.MASK_TIMELESS, "reference"),
    output_features = (FeatureType.META_INFO, "valid_reference"),
    map_function = checker_waterdata
)

Both the **validity criterions** have to be **merged to one**.
This will be done using the [ZipFeatureTask](https://eo-learn.readthedocs.io/en/latest/_m'odules/eolearn/core/core_tasks.html#ZipFeatureTask).

In [ ]:
def zipper(*arrays):
    return bool(np.all([np.all(array) for array in arrays]))
task_zip = ZipFeatureTask(
    input_features = [(FeatureType.META_INFO, "valid_data"),
                      (FeatureType.META_INFO, "valid_reference")],
    output_feature = (FeatureType.META_INFO, "valid"),
    zip_function = zipper
)

The **mask of our reference** will be taken into account while learning: during optimization and for the metrics of valid EOPatches.
The NDWI calculated by Sentinel-2 images is only reasonable on areas where no clouds occur.
That is, we mask these out.
Both, the **dataMask and the cloudMask will be joined** by the [JoinMasksTask](https://eo-learn.readthedocs.io/en/latest/_modules/eolearn/mask/masking.html#JoinMasksTask).

In [ ]:
task_reference_mask = JoinMasksTask(
    input_features = [(FeatureType.MASK_TIMELESS,"dmask_reference"),(FeatureType.MASK_TIMELESS,"cmask_reference")],
    output_feature = (FeatureType.MASK_TIMELESS, "mask_reference"),
    join_operation = 'and'
)

## Merging and Saving
**Different timestamps of features are not supported** by eo-learn, for now.
Accordingly, we have to define our **input and our output separately** as they live in different intervals and **join them together afterwards** using the [Merge-EOPatches-Task](https://eo-learn.readthedocs.io/en/latest/eolearn.core.eodata.html#eolearn.core.eodata.EOPatch.merge).

In [ ]:
#%% merge input and reference
task_merge = MergeEOPatchesTask(
    time_dependent_op = "concatenate", 
    timeless_op = "concatenate"
)

Afterwards, **only valid EOPatches are saved** using the [Save-Valid-Task]() based on the citerions regarding the input data availability and fraction of water in the reference data.
Note the **compression** keyword - if not set, the memory consumption may get really large!

In [ ]:
#%% save EOPatches
task_save = SaveValidTask(
    feature_to_check = (FeatureType.META_INFO, "valid"),
    path = config["dir_data"],
    filesystem = None,
    config = config["SHconfig"],
    overwrite_permission = OverwritePermission.OVERWRITE_PATCH,
    compress_level = 2
)

# Workflow
Now, we can define a workflow bringing everything together
- ### Input
>- task_data
>- task_pick
>- task_tdigest
>- task_data_check

- ### Reference
>- task_reference
>- task_reference_mask
>- task_reference_check

- ### Merging and Saving
>- task_merge
>- task_zip
>- task_save

## Define Nodes
Let's initialise the nodes we will use for our workflow afterwards.

In [ ]:
#%% input nodes
node_data = EONode(
    task = task_data,
    inputs = [],
    name = "load Sentinel-1 data"
)
node_pick = EONode(
    task = task_pick,
    inputs = [node_data],
    name = "pick closest observation to reference"
)
node_tdigest = EONode(
    task = task_tdigest,
    inputs = [node_pick],
    name = "compute T-Digest"
)
node_data_check = EONode(
    task = task_data_check,
    inputs = [node_tdigest],
    name = "check data for completeness"
)

#%% reference nodes
node_reference = EONode(
    task = task_reference,
    inputs = [],
    name = "load Sentinel-2 based reference"
)
node_reference_mask = EONode(
    task = task_reference_mask,
    inputs = [node_reference],
    name = "merge reference masks to one"
)
node_reference_check = EONode(
    task = task_reference_check,
    inputs = [node_reference_mask],
    name = "check reference for completeness"
)

#%% merging and saving nodes
node_merge = EONode(
    task = task_merge,
    inputs = [node_data_check,node_reference_check],
    name = "merge data and reference EOPatches"
)
node_zip = EONode(
    task = task_zip,
    inputs = [node_merge],
    name = "zip checking criterions"
)
node_save = EONode(
    task = task_save,
    inputs = [node_zip],
    name = "save valid EOPatch"
)

## Define Workflow
Now, we finally can define a workflow based on our tasks and nodes.
We could either put every single node in the constructor using a list or define our whole workflow by just the last node: `node_save`.

In [ ]:
workflow = EOWorkflow.from_endnodes(node_save)
#workflow.dependency_graph()

# Workflow Arguments
Now it's time to download the data.
Therefore, we have to define workflow arguments, both temporal and spatially.
Note that we only want to download the data which does not exist on our device.
Hence, we check for existence first and assign arguments afterwards.

In [ ]:
workflow_args = []
for _ in ["train","validation","test"]:
    print(_)
    bbox_list_ = bbox_lists[_]
    for i in np.random.randint(0,len(bbox_list_),40 if _=="train" else 10):##########################range(len(bbox_list_)):
        print(f"\r{i+1}/{len(bbox_list_)}",end="\r")
        #%%% query available timestamps
        timestamps_ = get_available_timestamps(
            bbox = bbox_list_[i], 
            data_collection = DataCollection.SENTINEL2_L1C, 
            time_interval = (config[f"start_{_}"],config[f"end_{_}"]), 
            time_difference = dt.timedelta(hours=12, seconds=0), 
            maxcc = config["maxcc"], 
            config = config["SHconfig"]
        )

        #%%% define corresponding arguments
        for timestamp_ in timestamps_:
            try:
                timestamps2_ = get_available_timestamps(
                    bbox = bbox_list_[i], 
                    data_collection = DataCollection.SENTINEL1_IW, 
                    time_interval = (timestamp_-config["datatimedelta"],timestamp_), 
                    time_difference = dt.timedelta(hours=0,seconds=0), 
                    maxcc = None, 
                    config = config["SHconfig"]
                )

                if timestamps2_:
                    dir_ = f"{_}/eopatch_{i}_{timestamp_.strftime(r'%Y-%m-%dT%H-%M-%S_%Z')}"
                    if not os.path.exists(os.path.join(config["dir_data"],dir_)):### and False: ### 
                        workflow_args.append(
                            {
                                node_data: {"bbox":bbox_list_[i],"time_interval":(timestamp_-config["datatimedelta"],timestamp_)},
                                node_reference: {"bbox":bbox_list_[i],"time_interval":timestamp_},
                                node_save: {"eopatch_folder":dir_}
                            }
                        )
            except Exception as e:
                print(e,(timestamp_-config["datatimedelta"],timestamp_))
    print()

print(f"Number of downloads: {len(workflow_args)}")

In [ ]:
workflow_args[-1]

# Executor
Our area of interest has been defined, our desired data has been defined, our workflow has been defined, our execution arguments have been defined, our executor...
This has to be done!

In [ ]:
#%% define executor
executor = EOExecutor(workflow, workflow_args, save_logs=True)

Let it run!
That may take a while...

In [ ]:
#%% run
executor.run(workers=config["threads"])
executor.make_report()

# Downloaded Data
After a long time, our executor finished with its work.
Let's **check** if there happened anything unexpected.

In [ ]:
failed_ids = executor.get_failed_executions()
if failed_ids:
    print(
        f"Execution failed EOPatches with IDs:\n{failed_ids}\n"
        f"For more info check report at {executor.get_report_path()}"
    )

Let's have a look how many `EOPatches` got stored to disk.

In [ ]:
print(f"Number of stored train EOPatches: {len(os.listdir(config['dir_train']))}")
print(f"Number of stored validation EOPatches: {len(os.listdir(config['dir_validation']))}")
print(f"Number of stored test EOPatches: {len(os.listdir(config['dir_test']))}")
print()
print(f"Number of downloads: {len(workflow_args)}")
print(f"Total number of EOPatches: {len(os.listdir(config['dir_train']))+len(os.listdir(config['dir_validation']))+len(os.listdir(config['dir_test']))}")

We finally made it!
Everything is ready for being used!

In [ ]:
print(FeatureType.DATA.ndim())
print(FeatureType.DATA_TIMELESS.ndim())
print(FeatureType.LABEL.ndim())
print(FeatureType.LABEL_TIMELESS.ndim())
print(FeatureType.MASK.ndim())
print(FeatureType.MASK_TIMELESS.ndim())
print(FeatureType.SCALAR.ndim())
print(FeatureType.SCALAR_TIMELESS.ndim())
print(FeatureType.VECTOR.ndim())
print(FeatureType.VECTOR_TIMELESS.ndim())